In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

In [ ]:
# Load California Housing dataset
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['PRICE'] = housing.target

In [ ]:
print("California Housing Dataset:")
print(f"Dataset shape: {df.shape}")
print("\nFeatures description:")
for i, feature in enumerate(housing.feature_names):
    print(f"{feature}: {housing.feature_names[i]}")
print("\nTarget variable: PRICE (median house value in $100,000s)")

In [ ]:
print("\nFirst 5 rows of the dataset:")
print(df.head())

In [ ]:
print("\nSummary statistics:")
print(df.describe())

In [ ]:
# Exploratory Data Analysis
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of California Housing Features')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

In [ ]:
# Split data into features and target
X = df.drop('PRICE', axis=1)
y = df['PRICE']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Base model with all features
def evaluate_model(X_train, X_test, y_train, y_test, feature_names, model_name="Model"):
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Training predictions and evaluation
    y_train_pred = model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = np.sqrt(train_mse)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Test predictions and evaluation
    y_test_pred = model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = np.sqrt(test_mse)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Cross-validation for more robust evaluation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    # Calculate adjusted R-squared for test set
    n = X_test.shape[0]  # number of observations
    p = X_test.shape[1]  # number of predictors
    test_adj_r2 = 1 - (1 - test_r2) * (n - 1) / (n - p - 1)
    
    print(f"\n--- {model_name} Evaluation ---")
    print(f"Features used: {', '.join(feature_names)}")
    print(f"Number of features: {len(feature_names)}")
    print(f"Training RMSE: {train_rmse:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"Cross-validation RMSE: {cv_rmse:.4f}")
    print(f"Training R²: {train_r2:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    print(f"Test Adjusted R²: {test_adj_r2:.4f}")
    
    # Return the coefficient values along with evaluation metrics
    coefficients = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': model.coef_
    }).sort_values(by='Coefficient', ascending=False)
    
    result = {
        'model': model,
        'feature_names': feature_names,
        'test_rmse': test_rmse,
        'test_r2': test_r2,
        'test_adj_r2': test_adj_r2,
        'coefficients': coefficients,
        'cv_rmse': cv_rmse
    }
    
    return result

# Evaluate base model with all features
base_result = evaluate_model(
    X_train_scaled, X_test_scaled, y_train, y_test, 
    X.columns, "Base Model (All Features)"
)

In [ ]:
# Check for multicollinearity using VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values('VIF', ascending=False)

print("\nVariance Inflation Factors (VIF) for feature multicollinearity:")
vif_df = calculate_vif(X_train)
print(vif_df)

In [ ]:
# --------------------------------------
# Forward Selection Implementation
# --------------------------------------
def forward_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    
    selected_features = []
    history = []
    
    while remaining_features:
        best_pvalue = float('inf')
        best_feature = None
        best_r2_adj = -float('inf')
        
        for feature in remaining_features:
            current_features = selected_features + [feature]
            X_current = X[current_features]
            
            # Add constant for statsmodels
            X_with_const = sm.add_constant(X_current)
            model = sm.OLS(y, X_with_const).fit()
            
            # Get the p-value for the newest feature
            if len(current_features) == 1:
                p_value = model.pvalues[1]  # Skip constant
            else:
                p_value = model.pvalues[-1]  # Last feature added
                
            r2_adj = model.rsquared_adj
            
            if p_value < best_pvalue and p_value < significance_level:
                best_pvalue = p_value
                best_feature = feature
                best_r2_adj = r2_adj
        
        if best_feature is None:
            break
            
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
        
        # For tracking the selection process
        X_selected = X[selected_features]
        X_with_const = sm.add_constant(X_selected)
        model = sm.OLS(y, X_with_const).fit()
        
        history.append({
            'step': len(selected_features),
            'feature_added': best_feature,
            'p_value': best_pvalue,
            'r2': model.rsquared,
            'r2_adj': model.rsquared_adj
        })
        
        print(f"Step {len(selected_features)}: Added {best_feature} (p-value: {best_pvalue:.6f}, Adj R²: {best_r2_adj:.6f})")
    
    return selected_features, history


In [ ]:
# --------------------------------------
# Backward Elimination Implementation
# --------------------------------------
def backward_elimination(X, y, significance_level=0.05):
    selected_features = list(X.columns)
    history = []
    
    while selected_features:
        X_selected = X[selected_features]
        X_with_const = sm.add_constant(X_selected)
        model = sm.OLS(y, X_with_const).fit()
        
        # Find the feature with the highest p-value
        p_values = model.pvalues[1:]  # Skip constant
        max_p_value = p_values.max()
        
        if max_p_value > significance_level:
            # Find the feature with the highest p-value
            feature_to_remove = selected_features[p_values.argmax()]
            selected_features.remove(feature_to_remove)
            
            # For tracking the elimination process
            history.append({
                'step': len(X.columns) - len(selected_features),
                'feature_removed': feature_to_remove,
                'p_value': max_p_value,
                'r2': model.rsquared,
                'r2_adj': model.rsquared_adj
            })
            
            print(f"Step {len(history)}: Removed {feature_to_remove} (p-value: {max_p_value:.6f}, Adj R²: {model.rsquared_adj:.6f})")
        else:
            break
    
    return selected_features, history


In [ ]:
# --------------------------------------
# Stepwise Selection Implementation
# --------------------------------------
def stepwise_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    
    selected_features = []
    history = []
    step_counter = 0
    
    while True:
        changed = False
        
        # Forward step
        best_pvalue = float('inf')
        best_feature = None
        best_r2_adj = -float('inf')
        
        for feature in remaining_features:
            current_features = selected_features + [feature]
            X_current = X[current_features]
            
            # Add constant for statsmodels
            X_with_const = sm.add_constant(X_current)
            model = sm.OLS(y, X_with_const).fit()
            
            # Get the p-value for the newest feature
            if len(current_features) == 1:
                p_value = model.pvalues[1]  # Skip constant
            else:
                p_value = model.pvalues[-1]  # Last feature added
                
            r2_adj = model.rsquared_adj
            
            if p_value < best_pvalue and p_value < significance_level:
                best_pvalue = p_value
                best_feature = feature
                best_r2_adj = r2_adj
        
        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            changed = True
            step_counter += 1
            
            history.append({
                'step': step_counter,
                'action': 'added',
                'feature': best_feature,
                'p_value': best_pvalue,
                'r2_adj': best_r2_adj
            })
            
            print(f"Step {step_counter}: Added {best_feature} (p-value: {best_pvalue:.6f}, Adj R²: {best_r2_adj:.6f})")
        
        # Backward step
        if len(selected_features) > 0:
            X_selected = X[selected_features]
            X_with_const = sm.add_constant(X_selected)
            model = sm.OLS(y, X_with_const).fit()
            
            # Find the feature with the highest p-value
            p_values = model.pvalues[1:]  # Skip constant
            max_p_value = p_values.max() if len(p_values) > 0 else 0
            
            if max_p_value > significance_level:
                # Find the feature with the highest p-value
                feature_idx = p_values.argmax()
                feature_to_remove = selected_features[feature_idx]
                selected_features.remove(feature_to_remove)
                remaining_features.append(feature_to_remove)
                changed = True
                step_counter += 1
                
                # For tracking the elimination process
                history.append({
                    'step': step_counter,
                    'action': 'removed',
                    'feature': feature_to_remove,
                    'p_value': max_p_value,
                    'r2_adj': model.rsquared_adj
                })
                
                print(f"Step {step_counter}: Removed {feature_to_remove} (p-value: {max_p_value:.6f}, Adj R²: {model.rsquared_adj:.6f})")
        
        if not changed:
            break
    
    return selected_features, history



In [ ]:
# Run Forward Selection
print("\n=== Forward Selection ===")
forward_features, forward_history = forward_selection(X_train, y_train)
print("\nSelected features:", forward_features)

# Evaluate Forward Selection model
forward_X_train = X_train[forward_features]
forward_X_test = X_test[forward_features]

# Standardize selected features
forward_X_train_scaled = scaler.fit_transform(forward_X_train)
forward_X_test_scaled = scaler.transform(forward_X_test)

forward_result = evaluate_model(
    forward_X_train_scaled, forward_X_test_scaled, y_train, y_test, 
    forward_features, "Forward Selection Model"
)